In [1]:

# find model from record of qlib
%load_ext autoreload
%autoreload 2
import sys
print(sys.path)
sys.path.insert(0, "/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib")

import qlib
print("qlib:",qlib.__path__)

qlib.init()

['/workspaces/Portfolio-Leveraging-Differentiable-Layer/experiment', '/opt/conda/envs/qlib/lib/python38.zip', '/opt/conda/envs/qlib/lib/python3.8', '/opt/conda/envs/qlib/lib/python3.8/lib-dynload', '', '/opt/conda/envs/qlib/lib/python3.8/site-packages', '/opt/conda/envs/qlib/lib/python3.8/site-packages/pyqlib-0.9.1.99-py3.8-linux-x86_64.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/protobuf-3.20.1-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/gym-0.26.2-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/Jinja2-3.0.3-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/filelock-3.13.1-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/dill-0.3.7-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/scikit_learn-1.3.2-py3.8-linux-x86_64.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/pymongo-3.7.2-py3.8-linux-x86_64.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/ruamel.yaml-0.18.5-py3.8.egg', '/opt/conda/en

[247257:MainThread](2023-11-21 10:59:07,519) INFO - qlib.Initialization - [config.py:417] - default_conf: client.
[247257:MainThread](2023-11-21 10:59:07,803) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[247257:MainThread](2023-11-21 10:59:07,804) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/codespace/.qlib/qlib_data/cn_data')}


## load the experiment you conducted before 

In [2]:
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
R.list_experiments()

/opt/conda/envs/qlib/lib/python3.8/site-packages/tqdm-4.66.1-py3.8.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'ExpQuickStartGRU': MLflowExperiment(id=466892454790696298, info={'class': 'Experiment', 'id': '466892454790696298', 'name': 'ExpQuickStartGRU', 'active_recorder': None, 'recorders': ['05eee8f1acc94d1280152e884d344dad']}),
 'Default': MLflowExperiment(id=0, info={'class': 'Experiment', 'id': '0', 'name': 'Default', 'active_recorder': None, 'recorders': []})}

## Find the recorder that corresponding to your prediction result/model

In [3]:
EXP_NAME = "ExpQuickStartGRU"
rid = "05eee8f1acc94d1280152e884d344dad"
rec = R.get_recorder(experiment_name=EXP_NAME,recorder_id=rid)

### load prediction result from your record

In [6]:
pred = rec.load_object("pred.pkl")
print(pred)

                          score
datetime   instrument          
2019-01-02 SH600000   -0.047405
           SH600004    0.051870
           SH600009   -0.111748
           SH600010    0.054693
           SH600011   -0.148266
...                         ...
2019-12-31 SZ300347    0.021195
           SZ300408    0.055576
           SZ300413   -0.127319
           SZ300433    0.025763
           SZ300498    0.000308

[73200 rows x 1 columns]


### run backtest based on pred

In [20]:
# import pickle
import pickle
# read corr_cvxpylayer.pkl

port_analysis_config={
    "strategy":{
        "class": "MarkowitzStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs":{
            "signal":"<PRED>",
            "verbose": True,
            "market":"csi300",
           "cov":corr_cvxpylayer,
           "gamma":1.44,
        },
    },
    'backtest':{
        "start_time": "2019-01-01",
        "end_time": "2019-12-31",
        "account": 100000000,
        "benchmark": "SH000300",
        "exchange_kwargs": {
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5
        }
    }
}
with R.start(experiment_name=EXP_NAME, recorder_id=rid, resume=True):
    # signal-based analysis
    rec = R.get_recorder()
    sar = SigAnaRecord(rec,ana_long_short=True)
    sar.generate()

    #  portfolio-based analysis: backtest
    par = PortAnaRecord(rec, port_analysis_config, "day")
    par.generate()

[247257:MainThread](2023-11-21 11:38:20,336) INFO - qlib.workflow - [exp.py:258] - Experiment 466892454790696298 starts running ...
[247257:MainThread](2023-11-21 11:38:20,345) INFO - qlib.workflow - [recorder.py:341] - Recorder 05eee8f1acc94d1280152e884d344dad starts running under Experiment 466892454790696298 ...
Exception in thread Thread-41:
Traceback (most recent call last):
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/client.py", line 297, in log_param
    self.store.log_param(run_id, param)
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 949, in log_param
    self._log_run_param(run_info, param)
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 955, in _log_run_param
    self._validate_new_param_value(
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 975, in _validate_new_param_value

{'IC': 0.01894293685644901,
 'ICIR': 0.13929638116337453,
 'Long-Avg Ann Return': 0.42262878036126494,
 'Long-Avg Ann Sharpe': 2.1398666055831317,
 'Long-Short Ann Return': 0.10887369003428536,
 'Long-Short Ann Sharpe': 1.8776074506257552,
 'Rank IC': 0.04259207242290374,
 'Rank ICIR': 0.3089255517858425}


[247257:MainThread](2023-11-21 11:38:30,573) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[247257:MainThread](2023-11-21 11:38:30,575) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[247257:MainThread](2023-11-21 11:38:30,578) WARNING - qlib.online operator - [exchange.py:226] - factor.day.bin file not exists or factor contains `nan`. Order using adjusted_price.
[247257:MainThread](2023-11-21 11:38:30,578) WARNING - qlib.online operator - [exchange.py:228] - trade unit 100 is not supported in adjusted_price mode.
[247257:MainThread](2023-11-21 11:38:32,230) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7faa075790a0>


cov: instrument  SH600000  SH600004  SH600006  SH600008  SH600009  SH600010  \
instrument                                                               
SH600000    0.801889  0.023673  0.010811 -0.067648  0.129266 -0.052452   
SH600004    0.023673  0.264494  0.067025  0.055287  0.071422 -0.000046   
SH600006    0.010811  0.067025  0.355768 -0.043275  0.009127 -0.020456   
SH600008   -0.067648  0.055287 -0.043275  0.859709  0.005216  0.027419   
SH600009    0.129266  0.071422  0.009127  0.005216  0.992537 -0.077483   
...              ...       ...       ...       ...       ...       ...   
SH603899   -0.001568 -0.001578  0.000132 -0.000858  0.001417 -0.002247   
SZ002607    0.000008  0.001096 -0.000897  0.005817  0.000631  0.001693   
SZ002916   -0.001777 -0.005268  0.001230 -0.007971 -0.003205 -0.002905   
SZ002958    0.000052  0.002960 -0.000471  0.003050  0.002143 -0.000400   
SZ300347   -0.001089 -0.004833  0.000833 -0.005403  0.000280 -0.001681   

instrument  SH600011  SH600015  

backtest loop:   0%|          | 0/244 [00:00<?, ?it/s]/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
[247257:MainThread](2023-11-21 11:38:32,393) INFO - qlib.ORStrategy_Sharpe - [signal_strategy.py:709] - Current:{'_settle_type': 'None', 'position': {'cash': 100000000, 'now_account_value': 100000000.0}, 'init_cash': 100000000} 
[247257:MainThread](2023-11-21 11:38:32,394) INFO - qlib.ORStrategy_Sharpe - [signal_strategy.py:710] - number of holding stocks: 300
[247257:MainThread](2023-11-21 11:38:32,397) INFO - qlib.ORStrategy_Sharpe - [signal_strategy.py:711] - Score & Weight:                score
instrument          
SH600000   -0.047405
SH600004    0.051870
SH600009   -0.111748
SH600010    0.054693
SH600011   -0.148266
...              ...
SZ300144   -0.038882
SZ300251   -0.046175
SZ300296    0.027935
SZ300408   -0.009128
SZ300433   -0.061523

[300 rows x 1 columns] [8.3020057

'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.001341
std                0.012507
annualized_return  0.319108
information_ratio  1.653894
max_drawdown      -0.140285
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean               0.000521
std                0.008271
annualized_return  0.123995
information_ratio  0.971754
max_drawdown      -0.135702
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean              -0.000881
std                0.008282
annualized_return -0.209776
information_ratio -1.641755
max_drawdown      -0.282497
'The following are analysis results of indicators(1day).'
        value
ffr  0.671812
pa   0.000000
pos  0.000000


In [9]:
R.end_exp()

In [21]:
# import pickle
import pickle
# read corr_cvxpylayer.pkl
with open('../data/corr_cvxpylayer.pkl', 'rb') as f:
    corr_cvxpylayer = pickle.load(f)
port_analysis_config={
    "strategy":{
        "class": "MarkowitzSelfExcludingStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs":{
            "signal":"<PRED>",
            "verbose": False,
            "market":"csi300",
           "cov":corr_cvxpylayer,
           "gamma":1.44,
        },
    },
    'backtest':{
        "start_time": "2019-01-01",
        "end_time": "2019-12-31",
        "account": 100000000,
        "benchmark": "SH000300",
        "exchange_kwargs": {
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5
        }
    }
}
with R.start(experiment_name=EXP_NAME, recorder_id=rid, resume=True):
    # signal-based analysis
    rec = R.get_recorder()
    sar = SigAnaRecord(rec,ana_long_short=True)
    sar.generate()

    #  portfolio-based analysis: backtest
    par = PortAnaRecord(rec, port_analysis_config, "day")
    par.generate()

[247257:MainThread](2023-11-21 11:40:53,550) INFO - qlib.workflow - [exp.py:258] - Experiment 466892454790696298 starts running ...
[247257:MainThread](2023-11-21 11:40:53,561) INFO - qlib.workflow - [recorder.py:341] - Recorder 05eee8f1acc94d1280152e884d344dad starts running under Experiment 466892454790696298 ...


Exception in thread Thread-45:
Traceback (most recent call last):
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/client.py", line 297, in log_param
    self.store.log_param(run_id, param)
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 949, in log_param
    self._log_run_param(run_info, param)
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 955, in _log_run_param
    self._validate_new_param_value(
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 975, in _validate_new_param_value
    raise MlflowException(
mlflow.exceptions.MlflowException: Changing param values is not allowed. Param with key='cmd-sys.argv' was already logged with value='/opt/conda/envs/qlib/lib/python3.8/site-packages/ipykernel_launcher.py --f=/home/codespace/.local/share/jupyter/runtime/kernel-v2-342QGGl6RadkAI2.json' for

{'IC': 0.01894293685644901,
 'ICIR': 0.13929638116337453,
 'Long-Avg Ann Return': 0.42262878036126494,
 'Long-Avg Ann Sharpe': 2.1398666055831317,
 'Long-Short Ann Return': 0.10887369003428536,
 'Long-Short Ann Sharpe': 1.8776074506257552,
 'Rank IC': 0.04259207242290374,
 'Rank ICIR': 0.3089255517858425}


[247257:MainThread](2023-11-21 11:41:04,412) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[247257:MainThread](2023-11-21 11:41:04,414) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[247257:MainThread](2023-11-21 11:41:04,416) WARNING - qlib.online operator - [exchange.py:226] - factor.day.bin file not exists or factor contains `nan`. Order using adjusted_price.
[247257:MainThread](2023-11-21 11:41:04,417) WARNING - qlib.online operator - [exchange.py:228] - trade unit 100 is not supported in adjusted_price mode.
[247257:MainThread](2023-11-21 11:41:05,745) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7faa06aa4d30>


cov: instrument  SH600000  SH600004  SH600006  SH600008  SH600009  SH600010  \
instrument                                                               
SH600000    0.801889  0.023673  0.010811 -0.067648  0.129266 -0.052452   
SH600004    0.023673  0.264494  0.067025  0.055287  0.071422 -0.000046   
SH600006    0.010811  0.067025  0.355768 -0.043275  0.009127 -0.020456   
SH600008   -0.067648  0.055287 -0.043275  0.859709  0.005216  0.027419   
SH600009    0.129266  0.071422  0.009127  0.005216  0.992537 -0.077483   
...              ...       ...       ...       ...       ...       ...   
SH603899   -0.001568 -0.001578  0.000132 -0.000858  0.001417 -0.002247   
SZ002607    0.000008  0.001096 -0.000897  0.005817  0.000631  0.001693   
SZ002916   -0.001777 -0.005268  0.001230 -0.007971 -0.003205 -0.002905   
SZ002958    0.000052  0.002960 -0.000471  0.003050  0.002143 -0.000400   
SZ300347   -0.001089 -0.004833  0.000833 -0.005403  0.000280 -0.001681   

instrument  SH600011  SH600015  

backtest loop:   0%|          | 0/244 [00:00<?, ?it/s]/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
backtest loop:   1%|          | 2/244 [00:00<00:13, 17.70it/s]/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
backtest loop:   2%|▏         | 4/244 [00:00<00:24,  9.76it/s]/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
/workspaces/Portfolio-Leveraging-Different

IndexError: list index out of range

In [17]:
R.end_exp()